# Model 1

- 목적 : input data 를 not, exp로 분류하고 일정 threshold 이하일 때에는 ood로 판단하는 모델을 학습시킨 후 model_1로 저장한다.    
- 학습 데이터 : 일상대화문, 비윤리대화문 중 intensity '2.0' 이상을 추출한 데이터중에 VIOLENCE라벨을 가지고있는 것들, dktc_exp
- 참고사항 : 

In [ ]:
!pip install keras-bert
!pip install keras-radam
!pip install wandb

# !pip install tensorflow==2.12.0

In [1]:
# 라이브러리 import 및 주요 라이브러리 버전 확인 
import wandb
from wandb.keras import WandbCallback

import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle
from sklearn.model_selection import train_test_split

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

import codecs
from tqdm import tqdm
import shutil

import json

import os
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# tf.logging.set_verbosity(tf.logging.ERROR)

from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam
from keras_radam.training import RAdamOptimizer

In [ ]:
import encoding_korbert as enc

## 학습에 쓸 데이터 로딩

In [2]:
daily_train_df = pd.read_csv("./data/daily_train.csv", usecols=['class','conversation']) #일상대화 train 병합 데이터
daily_val_df = pd.read_csv("./data/daily_val.csv", usecols=['class','conversation']) #일상대화 val 병합 데이터

In [ ]:
dktc_exp_df = pd.read_csv("./data/dktc_exp.csv", usecols=['class','conversation'])
immoral_df = pd.read_csv("./data/immoral_over_2_v.csv", usecols=['class','conversation'])

In [6]:
merged_exp_df = pd.concat([dktc_exp_df, immoral_df], axis=0)
merged_df = merged_exp_df.reset_index(drop=True)

In [7]:
model_1_exp_train_df, model_1_exp_val_df = train_test_split(merged_exp_df, test_size=0.2)

In [ ]:
# # 샘플링 코드

# daily_train_df =daily_train_df.sample(n=50, random_state=1004)
# daily_val_df =daily_val_df.sample(n=10, random_state=1004)

# model_1_exp_train_df =model_1_exp_train_df.sample(n=50, random_state=1004)
# model_1_exp_val_df =model_1_exp_val_df.sample(n=10, random_state=1004)

In [8]:
model_1_train_df = pd.concat([daily_train_df, model_1_exp_train_df], join="inner")
model_1_train_df.reset_index(drop="index", inplace=True)

In [9]:
model_1_val_df = pd.concat([daily_val_df, model_1_exp_val_df], join="inner")
model_1_val_df.reset_index(drop="index", inplace=True)

In [ ]:
for i in range(10):
    model_1_train_df = model_1_train_df.sample(frac=1).reset_index(drop=True) #많이 섞기

for i in range(10):
    model_1_val_df = model_1_val_df.sample(frac=1).reset_index(drop=True)#많이 섞기

In [ ]:
model_1_train_df["class"] = model_1_train_df["class"].astype('category')
model_1_train_df["class"] = model_1_train_df["class"].cat.codes # 0:exp, 1:not

model_1_val_df["class"] = model_1_val_df["class"].astype('category')
model_1_val_df["class"] = model_1_val_df["class"].cat.codes # 0:exp, 1:not

## Encoding

In [ ]:
tokenizer = enc.tokenizer

In [ ]:
train_x, train_y = enc.load_data(model_1_train_df)
val_x, val_y = enc.load_data(model_1_val_df)

### 모델 파인튜닝

In [ ]:
wandb.login(key="fff4dcdf86063b9dafa0296b4abaeb7d3639da7d")

### init_config

In [ ]:
default_config = {
#     "n_channel_1" : 32,
#     "n_channel_2" : 64,
#     "n_dense" : 1024,
    "learning_rate" : 1e-4,
    "epochs" : 5,
    "batch_size" : 16,
    "weight_decay" : 0.025,
    "optimizer" : "radam",
    "loss" : "binary_crossentropy",
    "metrics" : ["accuracy"]
}

### sweep_config

In [ ]:
sweep_config = {
    "metric": {"name": "val_loss", "goal": "minimize"},
    "method": "random",
    "parameters": {
        "optimizer" : {
            'value' : 'radam'
            },
        "batch_size" : {
            "values" : [8, 16] # OOM 에러시 바꿔주세용
            },
        "learning_rate" : {
            "min" : 1e-5,
            "max" : 1e-4 # 0에 가까울 수록
            },
        "weight_decay" : {
            "values" : [0.025, 0.001]
        },
        "epochs" : {
            "distribution" : "int_uniform",
            "min" : 3,
            "max" : 5
            }
                    
        }
    }

In [14]:
pretrained_path ="./bert" #상대경로 잡기

SEQ_LEN = enc.SEQ_LEN

config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')

In [15]:
def build_model():
    model = load_trained_model_from_checkpoint(config_path,
                                                checkpoint_path,
                                                training=True,
                                                trainable=True,
                                                seq_len=SEQ_LEN)
    
    inputs = model.inputs[:2]
    dense = model.layers[-3].output
    
    outputs = keras.layers.Dense(1, activation='sigmoid', 
                               kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)

    bert_model = keras.models.Model(inputs, outputs)

    return bert_model

In [ ]:
# def set_optimizer():
#     if config.optimizer == "radam":
#         return RAdamOptimizer(learning_rate=config.learning_rate, 
#                               weight_decay=config.weight_decay)

In [ ]:
# WandbCallback's labels
CLASS_NAMES = ["위협 대화", "일상 대화"] # 0:exp, 1:not

In [ ]:
path= './saved'

layer_num = 12

In [ ]:
def keras_callbacks():                       # 날짜 바꿔주세용
    CK = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(path, "model_1_weights_0419.{epoch:02d}-{val_loss:.2f}.h5"),
                                              monitor='val_loss',
                                              save_best_only = True,
                                              save_weights_only = True)

    ES = tf.keras.callbacks.EarlyStopping(patience=1)
    
    return CK, ES

In [ ]:
def train():
    global default_config

    # wandb.init & config ok
    run = wandb.init(project = 'model_1_0419', # 날짜만 변경해주세요
                    entity = "m05",
                    config = default_config)
    config = wandb.config

    # Model ok
    model = build_model()    
    
    # optimizer 함수 불러오기 ok
#     optimizer = set_optimizer()
    if config.optimizer == "radam":
        OPTIMIZER = RAdamOptimizer(learning_rate=config.learning_rate, 
                              weight_decay=config.weight_decay)
    
    # model.compile ok
    model.compile(optimizer = OPTIMIZER,
                  loss = config.loss,
                  metrics= config.metrics)
    
    #keras_callbacks ok
    CK, ES = keras_callbacks()
    
    # model.fit
    model.fit(train_x, train_y,
              epochs = config.epochs,
              batch_size = config.batch_size,
              validation_data = (val_x, val_y),
              callbacks = [CK, ES , 
                           WandbCallback(training_data = (train_x[:30], train_y[:30]),
                                         validation_data = (val_x[:30], val_y[:30]),
                                        labels = CLASS_NAMES)])
                                        
    # weight 모델 저장
    tf.keras.models.save_model(model, 'model_1')
    
#     # evaluate test
#     test_loss, test_accuracy = model.evaluate(test_x, test_y, verbose=2)
    
#     # wandb's 테이블쪽 column 추가하기
#     wandb.log({"Test Accuracy Rate: " : round(test_accuracy * 100, 2),
#                "Test Error Rate: " : round((1 - test_accuracy) * 100, 2)})
    
    run.finish() # run 종료

### model.fit 실행

In [ ]:
sweep_id = wandb.sweep(sweep_config,
                       entity = 'm05', 
                       project = 'model_1_0419') # 날짜만 변경해주세요

# run the sweep
wandb.agent(sweep_id,
            count=5, # 몇회 돌릴지 선택해주세요
            function=train)

---
---

### model summary 확인

In [16]:
model_1= build_model()
model_1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 256, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 256, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
#레이어의 trainable 출력해서 확인하기

for layer in model_1.layers:
    if 'Encoder' in layer.name:
        if 'layer_0' in layer.name:
            if layer.trainable == False:
                print(f'{layer.name} is frozen, 고정됨')
            else:
                print(f'{layer.name} is not frozen, 고정안됨')
        elif layer.trainable == False:
            print(f'{layer.name} is frozen, 고정됨')
        else:
            print(f'{layer.name} is not frozen, 고정안됨')

In [ ]:
# for layer in model_1.layers[:layer_num]:
#     layer.trainable = False # 층을 선택하여 freeze 하기.